In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [2]:
file_path = Path('factor_df.csv')
df = pd.read_csv(file_path)
df.head()

,regionname,city,Incr_Per,date,Incr_type,Year,month_date_yyyymm,postal_code,zip_name,active_listing_count_yy,median_days_on_market_yy,new_listing_count_yy,pending_ratio_yy
0,94109,San Francisco,0.011728,202008,Incr,2020,202007,94109,"san francisco, ca",0.7536,-0.0118,0.5000,-0.2405
1,94501,Alameda,0.036315,202008,Incr,2020,202007,94501,"alameda, ca",-0.2143,0.0909,-0.3333,0.5931
2,94558,Napa,0.021612,202008,Incr,2020,202007,94558,"napa, ca",-0.2304,-0.0571,0.0000,0.2472
3,94536,Fremont,0.010591,202008,Incr,2020,202007,94536,"fremont, ca",-0.5620,-0.1905,-0.1667,0.7942
4,95035,Milpitas,0.108590,202008,Huge_Incr,2020,202007,95035,"milpitas, ca",-0.4862,-0.3235,-0.1765,0.2946


In [3]:
df=df.dropna()

df.head()

,regionname,city,Incr_Per,date,Incr_type,Year,month_date_yyyymm,postal_code,zip_name,active_listing_count_yy,median_days_on_market_yy,new_listing_count_yy,pending_ratio_yy
0,94109,San Francisco,0.011728,202008,Incr,2020,202007,94109,"san francisco, ca",0.7536,-0.0118,0.5000,-0.2405
1,94501,Alameda,0.036315,202008,Incr,2020,202007,94501,"alameda, ca",-0.2143,0.0909,-0.3333,0.5931
2,94558,Napa,0.021612,202008,Incr,2020,202007,94558,"napa, ca",-0.2304,-0.0571,0.0000,0.2472
3,94536,Fremont,0.010591,202008,Incr,2020,202007,94536,"fremont, ca",-0.5620,-0.1905,-0.1667,0.7942
4,95035,Milpitas,0.108590,202008,Huge_Incr,2020,202007,95035,"milpitas, ca",-0.4862,-0.3235,-0.1765,0.2946


In [4]:
len(df)

484

In [5]:
df.loc[(df['Incr_type']=='Huge_Incr'), 'Incr_type'] = "Incr"
df.loc[(df['Incr_type']=='Huge_Decr'), 'Incr_type'] = "Decr"

In [6]:
X = df.copy()
X = X.drop(["regionname", "city", "Incr_Per", "date","Year","month_date_yyyymm","postal_code","zip_name","Incr_type","active_listing_count_yy","new_listing_count_yy"],axis=1)

X.head()

,median_days_on_market_yy,pending_ratio_yy
0,-0.0118,-0.2405
1,0.0909,0.5931
2,-0.0571,0.2472
3,-0.1905,0.7942
4,-0.3235,0.2946


In [7]:
y = df.Incr_type

In [8]:
X.dtypes

median_days_on_market_yy    float64
pending_ratio_yy            float64
dtype: object

In [9]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1)

In [10]:

# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)
Counter(y_resampled)

Counter({'Incr': 288, 'Decr': 288})

In [11]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [12]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6663366336633663

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[12,  8],
       [27, 74]], dtype=int64)

In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

       Decr       0.31      0.60      0.73      0.41      0.66      0.43        20
       Incr       0.90      0.73      0.60      0.81      0.66      0.45       101

avg / total       0.80      0.71      0.62      0.74      0.66      0.44       121

